# Mu vs 1Ar39
We see that the cnn model struggles in prediction of mu vs 1ar39.
The idea is to train a single model for each class of ar39, in order to later combine their prediction in a final outcome (ensemble).

In this notebook, we propose simple features on group of slices (_quadrant_) to see if with a restricted number of features, a linear model can be trained. Let see...

In [1]:
import os, numpy as np, pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [25, 15]
plt.rcParams.update({'font.size': 18})

In [2]:
def stddev_idslices_fun(row):
    # given an array of aquisitions, it return the std of activated slices
    # it create a populaion of slice ids, and compute stddev on them
    id_population = [item for id_list in [[i] * int(row[i]) for i in range(len(row))] for item in id_list]
    if id_population:    # check if the list of slice ids is not empty
        return np.std(id_population)
    else:
        return -1

In [3]:
def produce_quadrant_features(df):
    # split aquisitions in 4 "shifted" quadrant
    nslices = 72
    quadrant_width = nslices // 2   # idea: spread is within 30 slices (it depends on the sampling trick)
    nshiftings = 4                  # arbitrary grain
    shift = nslices // nshiftings   # derived
    df = df.iloc[:, 2:nslices+2]   # skip the first two fields (edep, pedetected)
    # create quadrants
    df_quadrants = []
    for i_shift in range(nshiftings):
        assert(i_shift*shift < nslices)
        quadrant = df.iloc[:, i_shift*shift:i_shift*shift + quadrant_width]
        if i_shift*shift + quadrant_width >= nslices:
            quadrant = pd.concat([quadrant, df.iloc[:, :i_shift*shift + quadrant_width - nslices]], axis=1)
        df_quadrants.append(quadrant)
    # compute stddev and meanPE for each quadrant
    for quadrant in df_quadrants:
        quadrant["stdslices"] = quadrant.apply(stddev_idslices_fun, axis=1)
        quadrant["meanpe"] = quadrant.apply(lambda row: row[:36].mean(), axis=1)
    # aggregate the features in a single dataset
    features = pd.DataFrame()
    for i, quadrant in enumerate(df_quadrants):
        features = pd.concat([features, quadrant[["stdslices", "meanpe"]]], axis=1)
    features.columns = ["{}_{}".format(col, j) for j in range(len(df_quadrants)) for col in ["stdslices", "meanpe"]]
    return features

In [4]:
filein = os.path.join("data", "Muons", "MarginalMuons_wt_0ar39_cut265PE.csv")
muons = pd.read_csv(filein, index_col=False)
#muons["std"] = muons.apply(lambda row: np.std([it for l in [[i] * int(row[2+i]) for i in range(72)] for it in l]), axis=1)

filein = os.path.join("data", "Ar39", "dataset6500", "Ar39_1Pileup_cut265PE_n6500.csv")
ar39_1 = pd.read_csv(filein, index_col=False)
#ar39_1["std"] = ar39_1.apply(lambda row: np.std([it for l in [[i] * int(row[2+i]) for i in range(72)] for it in l]), axis=1)

print("[Info] Loaded {} low-energy muons, {} single Ar39".format(len(muons), len(ar39_1)))

[Info] Loaded 6473 low-energy muons, 6500 single Ar39


In [12]:
muon_features = produce_quadrant_features(muons)
ar39_1_features = produce_quadrant_features(ar39_1)

/home/luigi/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/luigi/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
from sklearn.linear_model import LogisticRegression
import sklearn.model_selection as model_selection
from sklearn.metrics import roc_curve, plot_roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_recall_curve, plot_precision_recall_curve
from sklearn.preprocessing import StandardScaler
# models
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix

In [ ]:
# train easy model logistic regression
muon_features["y"] = 1
ar39_1_features["y"] = 0
data = pd.concat([muon_features, ar39_1_features], axis=0)
X, y = data.iloc[:, :-1], data.iloc[:, -1]
X = StandardScaler().fit_transform(X)
X = np.array(X)
y = np.array(y)
#to_bytes_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.8, random_state=0)

In [ ]:
names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", 
         "Decision Tree", "Random Forest", "AdaBoost",
         "Naive Bayes", "Logistic Regression"]
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    AdaBoostClassifier(),
    GaussianNB(),
    LogisticRegression()]

In [ ]:
import time
from sklearn.model_selection import KFold

model_template   = "[Info] Model: {}"
result_template  = "[Info] Complete training in {:.3f} seconds.\n"
result_template += "[Result] Score: {:.3f}, Accuracy: {:.3f}, Precision: {:.3f}, Recall: {:.3f}"
cv_template      = "[Detail] Confusion Matrix:\n"
cv_template     += "\tAr39\tMu\n"
cv_template     += "\tPred\tPred\n"
cv_template     += "----------------------\n"
cv_template     += "Ar39\t{}\t{}\n"
cv_template     += "Mu\t{}\t{}\n"
cv_template     += "----------------------\n"
# Start kfold cross-validation
n_folds = 10
kf = KFold(n_splits=n_folds, shuffle=True, random_state=0)
evaluations = []
for train_ids, test_ids in kf.split(X):
    X_train, X_test = X[train_ids], X[test_ids]
    y_train, y_test = y[train_ids], y[test_ids]
    # iterate over classifiers
    iteration_evaluations = []
    for name, clf in zip(names, classifiers):
        print(model_template.format(name))
        start = time.time()
        clf.fit(X_train, y_train)
        train_time = time.time() - start
        score = clf.score(X_test, y_test)
        y_pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        print(result_template.format(train_time, score, accuracy, precision, recall))
        #print(cv_template.format(tn, fp, fn, tp))
        iteration_evaluations.append({'accuracy': accuracy, 'precision': precision, 'recall': recall})
    evaluations.append(iteration_evaluations)

In [ ]:
accuracies = np.zeros(len(classifiers))
precisions = np.zeros(len(classifiers))
recalls = np.zeros(len(classifiers))
for iteration in evaluations:
    for i in range(len(classifiers)):
        accuracies[i] += iteration[i]["accuracy"]
        precisions[i] += iteration[i]["precision"]
        recalls[i] += iteration[i]["recall"]

accuracies /= len(evaluations)
precisions /= len(evaluations)
recalls /= len(evaluations)

In [ ]:
plt.bar(np.arange(len(names)), precisions, label="Purity (Precision)")
plt.bar(np.arange(len(names)), recalls, bottom=precisions, label="Efficiency (Recall)")
plt.xticks(np.arange(len(names)), names)
plt.legend()
plt.title("Purity/Efficiency averaged over 10-Fold Cross-Validation")
plt.xlabel("Models")
plt.ylabel("Stacked Purity,Efficiency")
plt.show()

In [ ]:
dtree = classifiers[3]

In [ ]:
plt.plot(np.arange(len(y_train)), y_train)

### Recap on features engineering:
- split the 72 readouts in 4 shifted quadrants: [0:36], [15:51], [36:72], [51:15]
- for each of this quadrant, compute: mean PE detected, std deviation of population of active slices
- the second one: population means [id] * PE[id]

In [ ]:
mask_std = ar39_1_perquad[["q1_std", "q2_std", "q3_std", "q4_std"]].max(axis=1)<7.1
mask_meanpe = ar39_1_perquad[["q1_meanpe", "q2_meanpe", "q3_meanpe", "q4_meanpe"]].max(axis=1)<2
print("TN: {}".format(len(ar39_1_perquad[(mask_std) & (mask_meanpe)])))
print("FP: {}".format(len(ar39_1_perquad[~(mask_std) | ~(mask_meanpe)])))

mask_std = muons_perquad[["q1_std", "q2_std", "q3_std", "q4_std"]].max(axis=1)>=7.1
mask_meanpe = muons_perquad[["q1_meanpe", "q2_meanpe", "q3_meanpe", "q4_meanpe"]].max(axis=1)>=2
print("FN: {}".format(len(muons_perquad[~(mask_std) | ~(mask_meanpe)])))
print("TP: {}".format(len(muons_perquad[(mask_std) & (mask_meanpe)])))

print("Tot Ar39: {}".format(len(ar39_1_perquad)))
print("Tot Mu: {}".format(len(muons_perquad)))

In [ ]:
filein = os.path.join("data", "Ar39", "dataset100000", "Ar39_1Pileup_cut265PE_n100000.csv")
all_ar39_1 = pd.read_csv(filein, index_col=False)
all_ar39_1_slices = all_ar39_1.iloc[:, 2:nslices+2]    # skip initial fields
all_ar39_1_quadrants = []
for i_shift in range(nshiftings):
    assert(i_shift*shift < nslices)
    quadrant = all_ar39_1_slices.iloc[:, i_shift*shift:i_shift*shift + quadrant_width]
    if i_shift*shift + quadrant_width >= nslices:
        quadrant = pd.concat([quadrant, all_ar39_1_slices.iloc[:, :i_shift*shift + quadrant_width - nslices]], axis=1)
    all_ar39_1_quadrants.append(quadrant)
for quadrant in all_ar39_1_quadrants:
    quadrant["stdslices"] = quadrant.apply(stddev_idslices_fun, axis=1)
    quadrant["meanpe"] = quadrant.apply(lambda row: row[:36].mean(), axis=1)

In [ ]:
all_ar39_1_features = pd.DataFrame()
for i, quadrant in enumerate(all_ar39_1_quadrants):
    all_ar39_1_features = pd.concat([all_ar39_1_features, quadrant[["stdslices", "meanpe"]]], axis=1)
all_ar39_1_features.columns = ["{}_{}".format(col, j) for j in range(len(all_ar39_1_quadrants)) for col in ["stdslices", "meanpe"]]

In [ ]:
all_ar39_1_features["y"] = 0

In [ ]:
X_test_ar39, y_test_ar39 = all_ar39_1_features.iloc[:, :-1], all_ar39_1_features.iloc[:, -1]
X_test_ar39 = StandardScaler().fit_transform(X_test_ar39)
X_test_ar39 = np.array(X_test_ar39)
y_test_ar39 = np.array(y_test_ar39)

In [ ]:
dtree.score(X_test_ar39, y_test_ar39)